<a href="https://colab.research.google.com/github/gm5000136/ISO-3166-Countries-with-Regional-Codes/blob/master/Water_Aid_workshop_2025_09_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the AI for Insight Workshop!

Today we're going to explore how AI can be your "strategic companion" - a powerful tool that helps you find new insights in your data and translate them into actionable business intelligence.

By the end of this workshop, you'll know how to:
- Use AI to discover patterns in messy, unstructured data
- Convert business questions into technical queries using AI
- Transform data insights into strategic recommendations

Let's dive in and see what AI can reveal about your work!


# Activity 1: The Discovery Sprint

**Goal:** Use AI to find hidden insights in unstructured data.

We're working with real WaterAid supporter survey responses - exactly the kind of unstructured data that's difficult to analyze manually but where AI can shine.

**The Challenge:** Instead of reading through 1,871 responses manually, we'll use AI to quickly identify themes, motivations, and insights that supporters are expressing.

**What You'll Learn:**
- How AI can process large volumes of text data quickly
- Different analytical approaches reveal different insights
- The importance of prompt engineering for quality results
- How to move from data to actionable insights

Let's start by setting up our analysis tools.

In [ ]:
import pandas as pd
import numpy as np
import os
import requests
from openai import OpenAI
import textwrap

# Download files from R2
api_key_url = "https://pub-349311c917e04ae9a03bab94c8354511.r2.dev/aiungeub44w8h9ff.txt"
supporter_csv_url = "https://pub-349311c917e04ae9a03bab94c8354511.r2.dev/supporter_survey_open_ends%202022.csv"
demographic_csv_url = "https://pub-349311c917e04ae9a03bab94c8354511.r2.dev/demographic_crosstabs_final.csv"

# Load API key
api_key_response = requests.get(api_key_url)
api_key = api_key_response.text.strip()
os.environ['OPENAI_API_KEY'] = api_key

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

# Download and save CSV files locally
supporter_csv_response = requests.get(supporter_csv_url)
with open('Supporter survey open ends 2022.csv', 'wb') as f:
    f.write(supporter_csv_response.content)

demographic_csv_response = requests.get(demographic_csv_url)
with open('demographic_crosstabs_cleaned.csv', 'wb') as f:
    f.write(demographic_csv_response.content)

print("✅ Data and API key loaded from Cloudflare R2!")

# print_wrapped, print_wrapped_preserve_structure and print_wrapped_stream make it easier to see the
# outputs from our LLM through this codebook
def print_wrapped(text, width=80):
    """Print text with word wrapping at specified width"""
    wrapped_text = textwrap.fill(text, width=width)
    print(wrapped_text)

def print_wrapped_preserve_structure(text, width=80):
    """Print text with wrapping while preserving line breaks and structure"""
    lines = text.split('\n')
    for line in lines:
        if line.strip():  # Non-empty lines
            wrapped = textwrap.fill(line, width=width)
            print(wrapped)
        else:  # Empty lines
            print()

print("Setup complete - ready to analyze supporter data")

## Phase 0: Load in real supporter data

We're using actual survey responses from WaterAid supporters who answered the question: *"With so many fantastic charities delivering great work, what is it that makes WaterAid one of the charities you choose to support?"*

This gives us authentic insights into donor motivations rather than synthetic examples. We'll sample 150 responses to balance comprehensive analysis with practical workshop timing.

In [ ]:
# Load and prepare WaterAid survey data
df = pd.read_csv('Supporter survey open ends 2022.csv')

# Clean column names
df.columns = ['why_support', 'topic_interests', 'experience_feedback', 'blank']

# Get usable text data
why_support_data = df['why_support'].dropna()
experience_data = df['experience_feedback'].dropna()

print(f"Loaded {len(df)} total survey responses")
print(f"Why support WaterAid: {len(why_support_data)} responses")
print(f"Experience feedback: {len(experience_data)} responses")

# Sample for analysis (150 responses as agreed)
sample_size = 150
why_sample = why_support_data.sample(min(sample_size, len(why_support_data)), random_state=42)

# Create analysis-ready format
feedback_data = "\n".join([f"Response {i+1}: {response}" for i, response in enumerate(why_sample)])

print(f"\nReady to analyze {len(why_sample)} responses about why supporters choose WaterAid")
print(f"\nSample response: {why_sample.iloc[0][:1000]}...")

## Phase 1: Basic Theme Analysis

Now we'll use AI to identify the most common themes in supporter responses. This baseline analysis helps us understand the primary patterns before we dig deeper with different analytical lenses.

**What to expect:** AI will process 150 supporter responses and identify recurring themes about why people choose to support WaterAid specifically.

In [ ]:
def analyze_feedback(feedback_text, custom_prompt=None):
    """
    Analyze supporter feedback using AI
    """
    if custom_prompt is None:
        prompt = """
        Analyze the following WaterAid supporter responses and identify the three most common themes.
        Respond with a clear bulleted list.

        Supporter responses:
        """ + feedback_text
    else:
        prompt = custom_prompt + "\n\nSupporter responses:\n" + feedback_text

    try:
        response = client.chat.completions.create(
            model="gpt-5",
            messages=[
                {"role": "system", "content": "You are a fundraising analyst specializing in supporter motivation analysis. Provide clear, actionable insights."},
                {"role": "user", "content": prompt}
            ]
            # Note: no streaming since in colab it has little benefit
        )

        # Get the complete response content
        full_response = response.choices[0].message.content

        return full_response

    except Exception as e:
        return f"Error: {str(e)}"

# Basic theme analysis (using data from previous cell)
print("Running AI analysis on WaterAid supporter feedback...")
print("=" * 50)
result = analyze_feedback(feedback_data)
formatted_result = result.replace('\n- ', '\n\n- ')

print("Main Themes:")
print_wrapped_preserve_structure(formatted_result)

## Phase 2: Supporter Segmentation Analysis

Not all supporters are motivated by the same factors. Let's see if AI can identify distinct supporter segments based on their response patterns.

**Why segment supporters?**
- Tailor communication strategies to different motivations
- Identify your most engaged supporter types
- Develop targeted cultivation approaches
- Understand potential conflicts between supporter groups

**What AI will identify?**

In [ ]:
# Supporter segmentation analysis

print("SUPPORTER SEGMENTATION")
print("=" * 50)

segmentation = analyze_feedback(feedback_data,
    """
    Based on these responses, identify 4 distinct types of WaterAid supporters.

    For each type:
    - Descriptive name (1 - 3 words)
    - Key characteristics (1 - 2 short sentences)
    - Primary motivation (1 sentence)
    - Best communication approach (1 sentence)
    - Three most illustrative quotes
    """)

print_wrapped_preserve_structure(segmentation)

 Hopefully you saw that it's created a set of distinct supporter archetypes with different characteristics, motivations, and communication preferences.

## Phase 3: Your Custom Analysis

Now it's your turn to dig deeper. Based on what you've discovered, what specific questions do you want to explore?

**Advanced Prompt Techniques:**
- **Assumption analysis**: "What assumptions about WaterAid do supporters reveal?"
- **Gap analysis**: "What do supporters NOT mention that might be significant?"
- **Strategic insights**: "What are your top 3 recommendations based on this feedback?"
- **Tension identification**: "What potential conflicts exist between different supporter views?"

**Tips for Better Results:**
- Be specific about what you want to discover
- Ask for actionable insights, not just descriptions
- Try "What if" scenarios based on the feedback
- Compare different supporter perspectives

Change the `your_prompt` variable below to explore whatever interests you most about your supporters.

In [ ]:
# Your turn to explore

# Change this prompt to investigate what interests you most
your_prompt = "What do supporters NOT mention that might be significant?"

print("CUSTOM ANALYSIS")
print("=" * 30)
print(f"Question: {your_prompt}")
print("-" * 50)

custom_result = analyze_feedback(feedback_data, your_prompt)
print_wrapped_preserve_structure(custom_result)

print("\n" + "=" * 50)
print("Try other prompts by changing 'your_prompt' above:")
print("- What do supporters NOT mention that might be significant?")
print("- What are supporters most proud of about supporting WaterAid?")
print("- How do supporters describe WaterAid's uniqueness?")
print("- What concerns or hesitations do supporters express?")

# Uncomment to try multiple analyses:
# print("\n" + "=" * 30)
# result2 = analyze_feedback(feedback_data, "What concerns do supporters have about WaterAid?")
# print("Supporter Concerns:")
# print(result2)

# Activity 2: Business translator
## Turning questions into actionable insights

**Goal:** Use AI to convert business questions into code that analyzes real WaterAid supporter data.

We're working with a WaterAid demographic dataset. This is cross-tabulated so that we can compare datapoints with one another e.g. we can ask, how many people have lived in a country where WaterAid works.

This demonstrates how AI can help fundraising and communications teams quickly analyze supporter data to make strategic decisions.

In [ ]:
# Load demographic data and create summary for AI
demo_df = pd.read_csv('demographic_crosstabs_cleaned.csv')

# Create a reusable data summary
def create_data_context():
    # Get all unique categories (same pool used in both columns)
    all_categories = (demo_df['answer_text'].unique().tolist())

    context = f"""Dataset: demographic_crosstabs_cleaned.csv - CROSSTAB ANALYSIS DATA

IMPORTANT: This is pre-aggregated crosstab data showing intersections between categories.

Structure:
- Each row shows intersection of TWO categories from the same pool
- comparison_dimension and answer_text both draw from identical category values
- count = number of people in BOTH categories
- percentage = percentage for this intersection
- No self-intersections (e.g. male+male doesn't exist)

Example: comparison_dimension="male", answer_text="I have lived/worked in one of the countries WaterAid works in", count=15
Means: 15 people are BOTH male AND have spent time in countries WaterAid works in.

AVAILABLE CATEGORIES (use exact strings for filtering):
{all_categories}

Total intersections: {len(demo_df)} rows
Columns: {list(demo_df.columns)}

Sample intersections:
{demo_df.head(5).to_string()}

TO ANALYZE: Use exact category strings above in pandas filters like:
demo_df[(demo_df['comparison_dimension'] == 'male') & (demo_df['answer_text'] == 'university support')]
"""
    return context

DATA_CONTEXT = create_data_context()
# print(DATA_CONTEXT)
print("📊 Data loaded and summarized for AI analysis")

## Set up our code generation
In colab (and any jupyter notebook) we need to run every cell in order to get them into context. Running below will just give a confirmation that the functions we need are now set.

In [ ]:
def generate_pandas_analysis_with_ai(question):
    """Generate pandas code to answer a business question about crosstab data"""
    prompt = f"""{DATA_CONTEXT}

Question: {question}

Generate clean Python pandas code to answer this question. Use the dataframe 'demo_df'.
Remember: This is crosstab data - look for intersections between categories.
Include comments explaining your filtering logic."""

    response = client.chat.completions.create(
        model="gpt-5",
        messages=[
            {"role": "system", "content": "You are a data analyst specialized in crosstab analysis. Generate executable pandas code with clear comments. Focus on finding intersections between demographic and behavioral categories."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

def run_business_analysis(question):
    """Generate and execute pandas analysis code"""
    print(f"🔍 Analyzing: {question}")
    print("=" * 50)

    try:
        # Generate code
        generated_code = generate_pandas_analysis_with_ai(question)

        # Show generated code
        print("Generated Code:")
        clean_code = generated_code.replace('```python', '').replace('```', '').strip()
        print_wrapped_preserve_structure(clean_code)
        print("\n" + "="*50)
        print("Results:")

        # Execute code with proper environment
        exec(clean_code, {'demo_df': demo_df, 'pd': pd, 'np': np, 'numpy': np, 're': __import__('re')})

    except Exception as e:
        print(f"❌ Execution Error: {e}")
        print("This is likely a code generation issue, not a data problem.")
        print("The AI-generated code may need debugging.")

# Confirm functions are loaded
print("✅ AI analysis functions loaded successfully!")
print("You can now use: run_business_analysis('your question here')")

### Test our code generation

In [ ]:
# Test with a more appropriate question for our crosstab data
test_question = "Show me the most common category intersections by count"

print("🧪 Testing AI Integration...")
try:
    test_code = generate_pandas_analysis_with_ai(test_question)
    print("✅ AI connection successful!")
    print("\nGenerated code preview:")
    preview = test_code.replace('```python', '').replace('```', '').strip()
    print(preview[:300] + "..." if len(preview) > 300 else preview)
except Exception as e:
    print(f"❌ AI connection failed: {e}")
    print("Please check your API key and internet connection")

### Some example questions

In [ ]:
# Helper to explore the data first
run_business_analysis("What categories contain the word 'age' or years?")

In [ ]:
# Examples that match our crosstab data structure
run_business_analysis("Compare how different age groups prefer to support WaterAid - show the top support methods for each age category")

In [ ]:
# Or if you have specific categories you know exist:
run_business_analysis("What motivates younger supporters (under 35) differently from older supporters (55+) to donate to WaterAid?")

## Workshop Wrap-Up 🎉

Congratulations! You've just experienced the complete AI-powered insight workflow:

**✅ What You've Accomplished:**

1. **Discovery Sprint** - Used AI to find themes in unstructured feedback data
2. **Business Translation** - Converted business questions into technical queries  
3. **Strategic Analysis** - Turned data results into actionable recommendations

**🚀 Key Takeaways:**

- **AI as Strategic Companion**: AI doesn't replace human judgment - it amplifies it
- **Prompt Engineering Matters**: How you ask questions determines the quality of insights
- **End-to-End Workflow**: From messy data to strategic recommendations in minutes

**💡 Next Steps:**

- Experiment with different prompts and questions
- Try this workflow with your own data  
- Think about where this could save time in your current processes

**Questions for Discussion:**
- Where else could this workflow be useful in your organization?
- What other types of unstructured data could benefit from AI analysis?
- How might this change how you approach data analysis tasks?

*Thank you for participating in the AI for Insight Workshop!*

In [ ]:
# Optional: Clean up and final thoughts

print("🎯 Workshop Complete!")
print("\n📈 What we accomplished:")
print("• Analyzed supporter feedback themes using AI")
print("• Generated SQL queries from business questions")
print("• Created strategic recommendations from data")
print("\n💡 Ready to apply these techniques to your real data!")

# Optional: Show workshop statistics
print(f"\n📊 Workshop Stats:")
print(f"• Analyzed {len(sample_feedback)} feedback entries")
print(f"• Queried database with {len(df)} supporter records")
print(f"• Generated strategic insights in under 10 minutes")

print("\n🚀 Keep experimenting with different prompts and data!")